In [37]:
import numpy as np
import pickle
import gc
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import pad_sequences
from keras.losses import categorical_crossentropy

In [3]:
# Load data
path = 'train_2p5M_struct.pkl'
with open(path, 'rb') as f:
    data_train = pickle.load(f).dataset


In [4]:
# Extract sequences and energies
sequences_train, energies_train, struct_train, hairpins_train = zip(*data_train)


# Free memory
del data_train
gc.collect()

0

In [5]:
# Convert sequences to n grams
def seq2ngrams(seqs, n=1):
    return np.array([[seq[i:i+n] for i in range(len(seq))] for seq in seqs], dtype=object)

maxlen_seq = 50
input_grams_train = seq2ngrams(sequences_train)

# Free memory
del sequences_train
gc.collect()

0

In [6]:
# Prepare for embedding
tokenizer_encoder = Tokenizer()
tokenizer_encoder.fit_on_texts(input_grams_train)
input_data_train = tokenizer_encoder.texts_to_sequences(input_grams_train)
input_data_train = pad_sequences(input_data_train, maxlen=maxlen_seq, padding='post')
n_words = len(tokenizer_encoder.word_index) + 1

tokenizer_decoder = Tokenizer(char_level=True)
tokenizer_decoder.fit_on_texts(struct_train)
struct_train = tokenizer_decoder.texts_to_sequences(struct_train)
struct_train = pad_sequences(struct_train, maxlen=maxlen_seq, padding='post')
struct_train = to_categorical(struct_train)


# Free memory
del input_grams_train
gc.collect()

0

In [7]:
@tf.keras.saving.register_keras_serializable()
def weighted_categorical_crossentropy(y_true, y_pred):
    class_weights = tf.constant([1.0, 1.0, 2.0, 2.0])
    weights = tf.reduce_sum(class_weights * y_true, axis=-1)
    unweighted_loss = categorical_crossentropy(y_true, y_pred)
    weighted_loss = unweighted_loss * weights
    return weighted_loss

In [8]:
# Load both models
full = tf.keras.models.load_model('full_2p5M.keras')
partial = tf.keras.models.load_model('partial_2p5M.keras')

In [38]:
# Load test data
path = 'data_test.pkl'
with open(path, 'rb') as f:
    test = pickle.load(f)

In [10]:
sequences_test, energies_test, struct_test, hairpins_test = zip(*test)

input_grams_test = seq2ngrams(sequences_test)

input_data_test = tokenizer_encoder.texts_to_sequences(input_grams_test)
input_data_test = pad_sequences(input_data_test, maxlen=maxlen_seq, padding='post')

struct_test = tokenizer_decoder.texts_to_sequences(struct_test)
struct_test = pad_sequences(struct_test, maxlen=maxlen_seq, padding='post')
struct_test = to_categorical(struct_test)

energies_test = np.asarray(energies_test)
hairpins_test = np.asarray(hairpins_test)

In [11]:
int_10_20 = []
int_21_30 = []
int_31_40 = []
int_41_50 = []
int_full = []

for i in test:
  seq_len = len(i[0])

  if seq_len >= 10 and seq_len <= 50:
    int_full.append(i)
  if seq_len >= 10 and seq_len <= 20:
    int_10_20.append(i)
  elif seq_len >= 21 and seq_len <= 30:
    int_21_30.append(i)
  elif seq_len >= 31 and seq_len <= 40:
    int_31_40.append(i)
  elif seq_len >= 41 and seq_len <= 50:
    int_41_50.append(i)

In [17]:
# Calculate metrics for each interval for the (mfe, hairpins) model

from sklearn.metrics import r2_score

for interval in [int_10_20, int_21_30, int_31_40, int_41_50, int_full]:
  print(f"Evaluating base model for interval with seq_len: {len(interval[0][0])} to {len(interval[-1][0])}")
  sequences_test, energies_test, struct_test, hairpins_test = zip(*interval)

  input_grams_test = seq2ngrams(sequences_test)

  input_data_test = tokenizer_encoder.texts_to_sequences(input_grams_test)
  input_data_test = pad_sequences(input_data_test, maxlen=maxlen_seq, padding='post')

  energies_test = np.asarray(energies_test)
  hairpins_test = np.asarray(hairpins_test)

  partial.evaluate(input_data_test, [energies_test, hairpins_test])

  pred = partial.predict(input_data_test)
  energies_pred = pred[0]
  hairpins_pred = pred[1]

  print(f'R2 for mfe: {r2_score(energies_test, energies_pred)}')
  print(f'R2 for hairpins: {r2_score(hairpins_test, hairpins_pred)}')
  print()

Evaluating base model for interval with seq_len: 10 to 20
172/172 [==============================] - 62s 312ms/step - loss: 0.0172 - MFE_loss: 0.0110 - Hairpins_loss: 0.0031
[0.017188794910907745, 0.010999493300914764, 0.003094649640843272]
172/172 [==============================] - 55s 316ms/step
R2 for mfe: 0.9861970262933883
R2 for hairpins: 0.8077363912507864

Evaluating base model for interval with seq_len: 21 to 30
157/157 [==============================] - 47s 300ms/step - loss: 0.1425 - MFE_loss: 0.0727 - Hairpins_loss: 0.0349
[0.14254246652126312, 0.07274679839611053, 0.034897830337285995]
157/157 [==============================] - 49s 314ms/step
R2 for mfe: 0.959739324079497
R2 for hairpins: 0.7254995573309282

Evaluating base model for interval with seq_len: 31 to 40
157/157 [==============================] - 50s 316ms/step - loss: 0.4641 - MFE_loss: 0.2384 - Hairpins_loss: 0.1128
[0.46406304836273193, 0.23843254148960114, 0.11281520873308182]
157/157 [======================

In [36]:
# Calculate metrics for each interval for the (mfe, hairpins, structure) model
from sklearn.metrics import classification_report
class_labels = {key:value for key,value in tokenizer_decoder.word_index.items()}

for interval in [int_10_20, int_21_30, int_31_40, int_41_50, int_full]:
  print(f"Evaluating full model for interval with seq_len: {len(interval[0][0])} to {len(interval[-1][0])}")
  sequences_test, energies_test, struct_test, hairpins_test = zip(*interval)

  input_grams_test = seq2ngrams(sequences_test)

  input_data_test = tokenizer_encoder.texts_to_sequences(input_grams_test)
  input_data_test = pad_sequences(input_data_test, maxlen=maxlen_seq, padding='post')

  struct_test = tokenizer_decoder.texts_to_sequences(struct_test)
  struct_test = pad_sequences(struct_test, maxlen=maxlen_seq, padding='post')
  struct_test = to_categorical(struct_test)

  energies_test = np.asarray(energies_test)
  hairpins_test = np.asarray(hairpins_test)
  full.evaluate(input_data_test, [energies_test, hairpins_test, struct_test])

  pred_seq = full.predict(input_data_test)[2]
  max_indices_pred = np.argmax(pred_seq, axis=2)
  max_indices_true = np.argmax(struct_test, axis=2)

  y_pred_flat = max_indices_pred.ravel()
  y_true_flat = max_indices_true.ravel()

  # Mask padding
  y_pred_flat_masked = y_pred_flat[y_pred_flat != 0]
  y_true_flat_masked = y_true_flat[y_true_flat != 0]

  report_dict = classification_report(y_true_flat_masked, y_pred_flat_masked, target_names=class_labels, output_dict=True)
  print(report_dict)
  print()

Evaluating full model for interval with seq_len: 10 to 20
172/172 [==============================] - 55s 317ms/step
{'.': {'precision': 0.9943668361656679, 'recall': 0.9887781954887218, 'f1-score': 0.9915646412380656, 'support': 53200}, '(': {'precision': 0.9792455381287182, 'recall': 0.9887372013651877, 'f1-score': 0.9839684804021466, 'support': 14650}, ')': {'precision': 0.9794691699871683, 'recall': 0.9899658703071672, 'f1-score': 0.9846895474759819, 'support': 14650}, 'accuracy': 0.9889818181818182, 'macro avg': {'precision': 0.984360514760518, 'recall': 0.9891604223870255, 'f1-score': 0.986740889705398, 'support': 82500}, 'weighted avg': {'precision': 0.98903619585347, 'recall': 0.9889818181818182, 'f1-score': 0.9889948972397535, 'support': 82500}}

Evaluating full model for interval with seq_len: 21 to 30
157/157 [==============================] - 50s 320ms/step
{'.': {'precision': 0.9758121670743696, 'recall': 0.9603122820437825, 'f1-score': 0.9680001814305802, 'support': 88894}